In [2]:
%load_ext line_profiler
from go_mcts import Go_MCTS
from mcts import MCTS
from bokeNet import ValueNet, policy_dist, features
import torch

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:
pi = PolicyNet()
checkpt = torch.load("/home/jupyter/BokeGo/policy_net_py/v0.2/policy_v0.2_2020-11-07_1.pt", map_location = torch.device("cpu"))
pi.load_state_dict(checkpt["model_state_dict"])
pi.eval()
tree = MCTS()
board = Go_MCTS(policy=pi)

NameError: name 'PolicyNet' is not defined

In [9]:
from torch.nn import MSELoss
import torch
v = ValueNet()
t = torch.randn(1,27,9,9)
loss = MSELoss()
loss(v(t), torch.Tensor(1))

tensor(1.2739e-05, grad_fn=<MseLossBackward>)

In [3]:
%lprun -f features features(board)

In [4]:
%lprun -f tree.do_rollout tree.do_rollout(board, 100)

In [7]:
%lprun -f tree._simulate tree._simulate(board)

In [7]:
%timeit tree._simulate(board)

215 ms ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%lprun -f board.find_random_child board.find_random_child()

In [7]:
%lprun -f tree._descend tree._descend(board, 100)

Timer unit: 1e-07 s

Total time: 6.92987 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\policy_net_py\mcts.py
Function: _descend at line 54

Line #      Hits         Time  Per Hit   % Time  Line Contents
    54                                               def _descend(self, node, total_visits):
    55                                                   "Return a path from root down to leaf via PUCT selection"
    56                                                   # Start at root (current position)
    57         1         27.0     27.0      0.0          path = [node]
    58                                                   while True:
    59                                                       # Is node a leaf?
    60         9        478.0     53.1      0.0              if node not in self.children or not self.children[node]:
    61                                                           # Heuristic: if node is "promising" (i.e. large # of visits),
    62                          

In [10]:
%timeit tree._descend(board, 100)

TypeError: _descend() takes 2 positional arguments but 3 were given

In [9]:
%lprun -f tree._puct_select tree._puct_select(board, 100)

Timer unit: 1e-07 s

Total time: 0.801725 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\policy_net_py\mcts.py
Function: _puct_select at line 92

Line #      Hits         Time  Per Hit   % Time  Line Contents
    92                                               def _puct_select(self, node, total_visits):
    93                                                   "Select a child of node with PUCT"
    94                                           
    95                                                   # Predictor + UCT (PUCT) variant used in AlphaGo
    96         1         32.0     32.0      0.0          def puct(n):
    97                                                       avg_reward = 0 if self.N[n] == 0 else self.Q[n] / self.N[n]
    98                                                       return avg_reward + (self.exploration_weight
    99                                                               * n.get_move_prob(n.last_move)
   100                                           

In [10]:
%timeit tree._puct_select(board, 100)

533 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit board.get_move_prob(board.last_move)

531 ns ± 28.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
import numpy as np

a = np.ones((1,9,9†))
b = np.zeros((7,9,9))
b

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0

In [33]:
np.vstack([a,a,a,a,a,b, b]).shape

(19, 9, 9)

In [1]:
import torch
from bokePolicy import ValueNet
a = torch.zeros(1,27,9,9)
v = ValueNet()
v(a)

tensor([[ 0.0166,  0.0280, -0.0086,  0.0041, -0.0002,  0.0384,  0.0605,  0.0204,
          0.0600, -0.0631, -0.0558,  0.0174, -0.0588, -0.0438,  0.0202,  0.0419,
          0.0196,  0.0327,  0.0251,  0.0177,  0.0401, -0.0292, -0.0021, -0.0147,
         -0.0123, -0.0314,  0.0484,  0.0227, -0.0088, -0.0082, -0.0211,  0.0673,
         -0.0208,  0.0353, -0.0216,  0.0548, -0.0634, -0.0608, -0.0361, -0.0214,
          0.0542, -0.0301, -0.0227, -0.0168,  0.0224, -0.0302, -0.0124,  0.0363,
          0.0415,  0.0465, -0.0353, -0.0517,  0.0183,  0.0004,  0.0069, -0.0564,
          0.0594, -0.0543, -0.0231, -0.0455,  0.0441,  0.0036,  0.0171, -0.0607,
          0.0152, -0.0083,  0.0028, -0.0071, -0.0234,  0.0289, -0.0333, -0.0299,
          0.0125,  0.0372,  0.0557, -0.0275,  0.0162, -0.0557, -0.0267,  0.0317,
          0.0043, -0.0326,  0.0077,  0.0388,  0.0013, -0.0345, -0.0192, -0.0390,
          0.0312,  0.0084,  0.0587,  0.0555, -0.0553,  0.0106,  0.0119,  0.0484,
         -0.0436,  0.0541,  

In [12]:
import pandas as pd
from torch.utils.data import DataLoader
from torch.nn import MSELoss
from bokeNet import NinebyNineGames, ValueNet, PolicyNet
data = NinebyNineGames("../data/hval.csv")
dataloader = DataLoader(data, batch_size = 16, shuffle = True)


In [15]:
ft, res = data[100]
v = ValueNet()
pi = PolicyNet()
v(ft.unsqueeze(0))

tensor([[-0.0001]], grad_fn=<ViewBackward>)

In [17]:
v = ValueNet()

loss = MSELoss()
for i, data in enumerate(dataloader, 0):
    inputs, res = data
    break
print(res)
loss(v(inputs), res.unsqueeze(1))


tensor([ 1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,
        -1.,  1.], dtype=torch.float64)


tensor(1.0091, dtype=torch.float64, grad_fn=<MseLossBackward>)

In [6]:
v(inputs).view(32)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<ViewBackward>)

In [18]:
x = v.conv(inputs)
x = x.view(-1, 128*9*9)
x = v.lin1(x)
v.tanh(x).view(-1)

tensor([-0.0158, -0.0157, -0.0147, -0.0148, -0.0191, -0.0151, -0.0167, -0.0171,
        -0.0152, -0.0153, -0.0158, -0.0181, -0.0158, -0.0156, -0.0157, -0.0141,
        -0.0157, -0.0152, -0.0151, -0.0150, -0.0150, -0.0176, -0.0157, -0.0182,
        -0.0155, -0.0177, -0.0141, -0.0151, -0.0161, -0.0153, -0.0162, -0.0163],
       grad_fn=<ViewBackward>)